## Simulation of foam front propagation on a primal mesh

The goal of this document is to document step-by step the development of the implementation for simulation of foam front propagation on a primal mesh.

-----




Solving the coupled equations 
\begin{align*}
    T_t + \| \nabla T \| = v
    S_t + \frac{\nabla T}{\| \nabla T \|} = \nabla S
\end{align*}
on a primary mesh

There are varies strategies. We can opt to solve the transient equations for 
$t \rightarrow \infty$, for speed with large time steps and for stability by an implicit method. This emulates the iteration.

The alternative is to solve the stationary equations,

We have the coupled system of equations:
\begin{align*}
   H_T(T, S) = \| \nabla T \| - v = 0 \\
   H_S(T, S) = \frac{\nabla T}{\| \nabla T \|} - \nabla S = 0
\end{align*}
which we write in compact form as
\begin{align*}
    U = (S,T) ,
    \qquad
    H(U) =
    \begin{pmatrix}
       H_1(T, S) \\
       H_2(T, S) 
    \end{pmatrix}
\end{align*}


One mayor design issue is the choice of the numerical Hamiltonian,
and there whether the weighting of the gradients is done before or after the evaluation of the Hamiltonian.

Given the gradients $G_1, G_2, G_3$ 
and ponderations $w_1, w_2, w_3$
to triangles attached to a node, con can
first average and then calculate the Hamiltonian of that average, 
or first calculate the Hamiltonian and then average the evaluated functions.

\begin{align*}
    H(G_1, G_2, G_3) = \begin{cases}
        \frac{w_1 H(G_1) + w_2 H(G_2) + w_3 H(G_3)}{w_1 + w_2 + w_3} \\
        H \Biggl( \frac{w_1 G_1 + w_2 G_2 + w_3 G_3}{w_1 + w_2 + w_3} \Biggr)
        \end{cases}
\end{align*}


Now, one choice for the weigts is to set them equal to the corresponding angle.

Another mayor design choice is the type of grid, whether it is rectangular, triangular or with more general poligons, and wether the discrete values are considered on a primary or on a dual grid.


We choose a discretizaion on a primary grid, i.e. the solution values are associated to the vertices. 
For each vertex there are two discrete variables.

For a well determined system of equations we need two equations associated to these vertices.

So, when assembling the system of equations we run over all vertices. In order to handle the geometry we need to acces the positions of each vertex.

The following code iterates over the vertices, giving the index and position of each.

In [1]:
import openmesh as om

mesh = om.read_trimesh('siete_nodos.off')

V=mesh.points()
Vx=V[:,0]
Vy=V[:,1]

for vh in mesh.vertices():
    nVertex = vh.idx()
    print(nVertex,': [ % .0f' % Vx[nVertex], ', % .0f' % Vy[nVertex],']')
    

0 : [  0 ,  0 ]
1 : [  0 ,  10 ]
2 : [  10 ,  0 ]
3 : [  10 ,  10 ]
4 : [  5 ,  0 ]
5 : [  6 ,  10 ]
6 : [  3 ,  5 ]
7 : [  8 ,  5 ]


For assembling the numerical Hamiltonian we need to access the neighboring vertices of a given vertex.

In [2]:
for vh in mesh.vertices():
    nVertex = vh.idx()
    
    vlist = []
    for vvh in mesh.vv(vh):
        idx = vvh.idx() # devuelve indice del VERTICE
        vlist.append(idx)
        
    print(nVertex,': ', vlist)  

0 :  [1, 6, 4]
1 :  [5, 6, 0]
2 :  [4, 7, 3]
3 :  [2, 7, 5]
4 :  [0, 6, 7, 2]
5 :  [3, 7, 6, 1]
6 :  [7, 4, 0, 1, 5]
7 :  [4, 6, 5, 3, 2]


A given vertex, this radiates by the vectors that connect to the neighbor vertices. We need to calculate the angles between these common edges. Therefore, first represent the vectors that connect the given vertex to its neighbors, and then we calculate the angles.

In [3]:
import numpy as np
np.set_printoptions(precision=2)

for vh in mesh.vertices():
    nVertex = vh.idx()
    
    vlist = []
    nlist = []
    for vvh in mesh.vv(vh):
        idx = vvh.idx() # devuelve indice del VERTICE
        vlist.append(idx)
        nlist.append(V[nVertex,:]-V[idx,:])
    nlist=np.array(nlist)
    print(nVertex,': \n', nlist,'\n')
    
    # for n in nlist:
    THETA=[]
    for k in range(len(nlist)):
            v1 = nlist[k-1,:]
            v2 = nlist[k,:]
            theta = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
            THETA.append(theta)

    THETA=np.array(THETA)            
    print('sum(', THETA,') = % .2f' % sum(THETA),'\n \n')
    # print(': [ % .0f' % THETA)
    
    # print(nVertex,': [ % .0f' % Vx[nVertex], ', % .0f' % Vy[nVertex],']')

0 : 
 [[  0. -10.   0.]
 [ -3.  -5.   0.]
 [ -5.   0.   0.]] 

sum( [1.57 0.54 1.03] ) =  3.14 
 

1 : 
 [[-6.  0.  0.]
 [-3.  5.  0.]
 [ 0. 10.  0.]] 

sum( [1.57 1.03 0.54] ) =  3.14 
 

2 : 
 [[  5.   0.   0.]
 [  2.  -5.   0.]
 [  0. -10.   0.]] 

sum( [1.57 1.19 0.38] ) =  3.14 
 

3 : 
 [[ 0. 10.  0.]
 [ 2.  5.  0.]
 [ 4.  0.  0.]] 

sum( [1.57 0.38 1.19] ) =  3.14 
 

4 : 
 [[ 5.  0.  0.]
 [ 2. -5.  0.]
 [-3. -5.  0.]
 [-5.  0.  0.]] 

sum( [3.14 1.19 0.92 1.03] ) =  6.28 
 

5 : 
 [[-4.  0.  0.]
 [-2.  5.  0.]
 [ 3.  5.  0.]
 [ 6.  0.  0.]] 

sum( [3.14 1.19 0.92 1.03] ) =  6.28 
 

6 : 
 [[-5.  0.  0.]
 [-2.  5.  0.]
 [ 3.  5.  0.]
 [ 3. -5.  0.]
 [-3. -5.  0.]] 

sum( [1.03 1.19 0.92 2.06 1.08] ) =  6.28 
 

7 : 
 [[ 3.  5.  0.]
 [ 5.  0.  0.]
 [ 2. -5.  0.]
 [-2. -5.  0.]
 [-2.  5.  0.]] 

sum( [0.92 1.03 1.19 0.76 2.38] ) =  6.28 
 



Naturrally, the number of angles is dynamic as it depends on the number of edges.


The sum of the angles is expected to be $2\pi$ but there is an exception or the corner points. The corner points are particular anyway, in this context (for a rectangular domain) they count with an angles $4/3$ but there is no handling of angles greater that $\pi$ or say these angles are represented as angles $\pi$.

Now, to each triangle attached to a point, beside the angle corresponds a gradient.

The vertex builds one triangle together with each of two subsequent neighbors.
The gradient can be calculated from the function values at these three points,
for example by the directional derivatives in the direction of the vectors of each of two connecting edges, 

Equivalently, the three involved points with their corresponding function values define a plan (linear function), that is defined by three parameters, where two of them correpond to the gradient.

So, there are (at least) two methods to calculate the gradient from the triangle points with their respective function values. The chosen method should be a matter of taste. Problems of numerical stability should not be a concern as we assume to deal with nice triangles that count with equilibrated angles.

Now, a major issuee is to get the gradient calculation not just done, but well organized, since the gradient calculation is a central task in the overall algorithm.

Naturally, the gradient calculation subroutine take the information on the triangle points as input and give the gradient as output, but it is not clear in which format the input should be given, whether the actual values are given, e.g. the effective point position and the the actual function values, another more general representation, like handles to the involve triangles.

The issue is that, for running any sort of optimization routine in order to solve an overall equation system, the solution variables of the equations cannot be global variables of the algoritm, but need to be handled as local variables that are transported through the subroutines. So, the function values of the nodes need to be input variables. Also we need to know at which points these function values are taken; it is sufficient to use their index values of the involved nodes are sufficent to access their corresponding positions, that are taken from a global register.
A good question is whether to handle the subroutine parameters individually

            (gx, gy) = getGradient(i1, i2, i3, u1, u2, u3)

or as lists (respective arrays)

            (g) = getGradient(I, U).

The advantage of the handling as lists 

            (g) = (gx, gy),    I = (i1, i2, i3),     U = (u1, u2, u3)

is their elegance in notation, but a concern is the eventual need to pack and unpack them, respective the correct handling of the lists as aglomerated data structure. For a starter we opt to use lists. (That is more involved but with the perspective to get a payout, and not postponing the need to lateron compactify the code.)

The general idea of the implementation is that the three points of the triangle together with the solution values define a plane that can be described by the model
\begin{align*}
    z = f(x,y) = ax + by + c,
\end{align*}
where $a, b, c$ are the parameters, $x,y$ are independent variables and $z$ is a dependent variable.
Namely for three points
\begin{align*}
    (x_1, y_1, z_1) \\
    (x_2, y_2, z_2) \\
    (x_3, y_3, z_3)
\end{align*}
inserted in the model we have the three equations
\begin{align*}
    z_1 = ax_1 + by_1 + c \\
    z_2 = ax_2 + by_2 + c \\
    z_3 = ax_3 + by_3 + c 
\end{align*}
Now, if the three three-dimensional point coordinates are known, then the three parameters $a, b, c$ can be determined y solving a system of three equations with the three parameters as unknowns. Since the model is linear, we have to solve a system of linear equations
\begin{align*}
    \begin{pmatrix}
        x_1 & y_1 & 1 \\
        x_2 & y_2 & 1 \\
        x_3 & y_3 & 1
    \end{pmatrix}    
    \begin{pmatrix}
        a \\ b \\ c
    \end{pmatrix}    
    =    
    \begin{pmatrix}
        z_1 \\ z_2 \\ z_3
    \end{pmatrix},
\end{align*}
that can be written in compact form as
\begin{align*}
    M g = b, \qquad
    M = 
    \begin{pmatrix}
        x_1 & y_1 & 1 \\
        x_2 & y_2 & 1 \\
        x_3 & y_3 & 1
    \end{pmatrix},
    \qquad
    b =
    \begin{pmatrix}
        z_1 \\ z_2 \\ z_3
    \end{pmatrix},
    \qquad 
    g =
    \begin{pmatrix}
        a \\ b \\ c
    \end{pmatrix}
\end{align*}
The matrix $M$ can be assembled by the point positions, the vector $b$ consists of the function values, and the searched vector $g$ contains the parameters $a,b,c$. So, the parameters can be calculated by solving the linear system of equations.

The gradient 
\begin{align*}
    \nabla f(x,y) = \Biggl( \frac{\partial f(x,y)}{\partial x} , \frac{\partial f(x,y)}{\partial y} \Biggr)
\end{align*}
of the plane described by the model is obtained by calculating the partial derivatives
\begin{align*}
    \frac{\partial f(x,y)}{\partial x} = \frac{\partial}{\partial x} ( ax + by + c) = a \\
    \frac{\partial f(x,y)}{\partial y} = \frac{\partial}{\partial y} ( ax + by + c) = b 
\end{align*}
This means that the gradient is given by the parameters as
\begin{align*}
    \nabla f(x,y) = (a, b)
\end{align*}









The alternative implementation based on the directional derivatives actually might also start with the three points.
Actually the directional derivatives migth be expressed in terms of the three points as
\begin{align*}
    x_a = x_2 - x_1, \qquad y_a = x_2 - x_1, \qquad, z_a = z_2 - z_1,
    x_b = x_3 - x_1, \qquad y_b = x_3 - x_1, \qquad, z_b = z_3 - z_1,
\end{align*}
i. e. the derivative in direction $(x_a, y_a)$ is $z_a$ and in direction $(x_b, y_b)$ it is $z_b$.

Now, an interpretation of the gradient in the coordinates of directional derivatives is that the gradient of the model of the plane that 
passes through the points $(x_a, y_a, z_a)$, $(x_b, y_b, z_b)$ and the origin $(0, 0, 0)$.

In terms of a 3-dimensional model
\begin{align*}
    z = a x + b y + c
\end{align*}
by inserting the origin we have that $c=0$, which is no surprise as the plan is passing through the origin.
The remaining model for the two remaining points is 
\begin{align*}
    z = a x + b y
\end{align*}
Inserting gives
\begin{align*}
    z_a = a x_a + b y_a, \\
    z_b = a x_b + b y_b,
\end{align*}
so we have two equations for the two still unknown parameters $a,b,$,
that can be written as
\begin{align*}
    \begin{pmatrix}
        x_a & y_a \\
        x_b & y_b
    \end{pmatrix}    
    \begin{pmatrix}
        a \\ b
    \end{pmatrix}    
    =    
    \begin{pmatrix}
        z_a \\ z_b
    \end{pmatrix},
\end{align*}
which in terms of the original points is
\begin{align*}
    \begin{pmatrix}
        x_2 - x_1 & y_2 - x_1 \\
        x_3 - x_1 & y_3 - x_1        
    \end{pmatrix}    
    \begin{pmatrix}
        a \\ b
    \end{pmatrix}    
    =
    \begin{pmatrix}
        z_2 - z_1 \\ 
        z_3 - z_1
    \end{pmatrix},
\end{align*}

Now we are ready for the implementation.




In [4]:
def getGradent(I, U):
    m = np.zeros((3, 3))
    b = np.zeros((3, 1))

    for k in range(3):
        # print(k)
        m[k,0] = V[I[k],0]
        m[k,1] = V[I[k],1]
        m[k,2] = 1
        b[k] = U[k]
                
    g = np.linalg.solve(m, b)    
    
    return g[0:2]

def getGradent2(I, U):
    m = np.zeros((2, 2))
    b = np.zeros((2, 1))

    for k in range(2):
        m[k,0] = V[I[k+1],0]-V[I[0],0]
        m[k,1] = V[I[k+1],1]-V[I[0],1]
        b[k] = U[k+1]-U[0]
                
    g = np.linalg.solve(m, b)    
    
    return g

I = [4, 6, 7]
U = [2, 2.5, 2.4]
g3 = getGradent(I, U)
g2 = getGradent2(I, U)
print('g2:', g2, '\n \n g3:',g3)

g2: [[-0.02]
 [ 0.09]] 
 
 g3: [[-0.02]
 [ 0.09]]


With the code for the calculation of the gradient implemented, how does it apply for the overall iteration? The gradients should be handled in parallel to the angles.  Therefore the input argument should be prepared.

The function values are generated from a predefined function. From a list of neighboring vertices we extract all index pairs of subsequent vertices, and generate list of index triples.





In [5]:
def getT(x,y):
    return x*(1+0.5*(1-y)**2)

T = getT(V[:,0],V[:,1])

print('vlist:',vlist,'; \t T:',T,'; \t T[vlist]: ', T[vlist], '\n\n')
for k in range(len(vlist)):
    I=[nVertex, vlist[k-1], vlist[k]]
    g = getGradent2(I, T[I])
    print('g[', I, '] = ', g.transpose())

    
    

vlist: [4, 6, 5, 3, 2] ; 	 T: [  0.    0.   15.  415.    7.5 249.   27.   72. ] ; 	 T[vlist]:  [  7.5  27.  249.  415.   15. ] 


g[ [7, 2, 4] ] =  [[ 1.5 12. ]]
g[ [7, 4, 6] ] =  [[9.  7.5]]
g[ [7, 6, 5] ] =  [[ 9. 39.]]
g[ [7, 5, 3] ] =  [[41.5 52. ]]
g[ [7, 3, 2] ] =  [[71.5 40. ]]


This procedure can be plugged into the iteration, but there is one detail: Does the calculation effectively allways work, or are there cases where the procedure breaks down, which is manifested by an error message on a singular matrix.

If the directial derivative are in (nearly) colinear direction, 
i.e. the three chosen points are positioned on the same line, the matrix becomes singular.
This is manifested by an either small angle $\theta \approx 0$ or an angle that is close to $\pi$. (Angles greater than $\pi$ do not occur by definition.)


Though we assume and have nice triangles in the interior of the domain, small angles effectively occur for points at the boundary, where angles $\theta = \pi$ appear naturally at boundaries that are straight lines. So, these cases need to be captured. But what should we do then, what is the interpretation? In this case there is just no gradient to calculate, to there is also no need to register the angle.

From a systematic point of view, as this happens with boundary vertices, should be an extra treatment? All nodes have a contribution to the overall equation system. However, the outer angles that cover an area outside the domain do not contribute to an average gradient. this also happens for corner points, where to neigborss are connected by an angle of $\theta = \pi/2$.

The triangle of points define a plane, and interestingly, the corresponding gradient should be rather similar to the other calculated gradients, so it won't affect the calculation much when included.

Anyway, all list excercise is done to obtain the information on angles and gradients, with the perspective to calculate the numerical Hamiltonian by a weighting rule. Since this information is dynamic, with varying length of the object, it is not too convenient to export this information to a superior subroutine, but should be done in place.

As intermedate excercise we calculate the average of the gradient for each considered central vertex as
\begin{align*}
    \bar{G} = \frac{\sum_{k=0}^{N} \theta_k g_k_}{\sum_{k=0}^{N}}
\end{align*}



In [6]:
import math

Glist = []
    
for vh in mesh.vertices():
    nVertex = vh.idx()
    
    vlist = []
    nlist = []
    
    for vvh in mesh.vv(vh):
        idx = vvh.idx() # devuelve indice del VERTICE
        vlist.append(idx)
        nlist.append(V[nVertex,:]-V[idx,:])
    nlist=np.array(nlist)
    print(nVertex,': \n', nlist,'\n')
    
    # for n in nlist:
    THETA=[]
    GRADIENT=[]
    for k in range(len(vlist)):
            v1 = nlist[k-1,:]
            v2 = nlist[k,:]
            theta = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
            I=[nVertex, vlist[k-1], vlist[k]]
            
            if (abs(theta)<math.pi-1e-3) and (abs(theta) > 1e-3) and (abs(theta-math.pi/2)>1e-3):
                g = getGradent2(I, T[I])
                THETA.append(theta)
                GRADIENT.append(g)
                print('g[', I, '] = ', g.transpose(), '\t \t theta[', I, '] = ', theta)
            else:
                print('\t \t \t \t \t theta[', I, '] = ', theta)

    THETA=np.array(THETA)  
    GRADIENT=np.array(GRADIENT)
    print('sum(', THETA,') = % .2f' % sum(THETA),'\n \n')
    print('GRADIENT:', GRADIENT,'\n \n') # = % .2f') #  % sum(GRADIENT),'\n \n')
    gav = GRADIENT.transpose().dot(THETA)/sum(THETA)
    Glist.append(gav)
  
    
Glist = np.array(Glist)
print('\n \n -------- List of average gradients ------- \n \n', Glist)


0 : 
 [[  0. -10.   0.]
 [ -3.  -5.   0.]
 [ -5.   0.   0.]] 

	 	 	 	 	 theta[ [0, 4, 1] ] =  1.5707963267948966
g[ [0, 1, 6] ] =  [[9. 0.]] 	 	 theta[ [0, 1, 6] ] =  0.5404195002705842
g[ [0, 6, 4] ] =  [[1.5 4.5]] 	 	 theta[ [0, 6, 4] ] =  1.0303768265243125
sum( [0.54 1.03] ) =  1.57 
 

GRADIENT: [[[9. ]
  [0. ]]

 [[1.5]
  [4.5]]] 
 

1 : 
 [[-6.  0.  0.]
 [-3.  5.  0.]
 [ 0. 10.  0.]] 

	 	 	 	 	 theta[ [1, 0, 5] ] =  1.5707963267948966
g[ [1, 5, 6] ] =  [[41.5 19.5]] 	 	 theta[ [1, 5, 6] ] =  1.0303768265243125
g[ [1, 6, 0] ] =  [[ 9. -0.]] 	 	 theta[ [1, 6, 0] ] =  0.5404195002705842
sum( [1.03 0.54] ) =  1.57 
 

GRADIENT: [[[41.5]
  [19.5]]

 [[ 9. ]
  [-0. ]]] 
 

2 : 
 [[  5.   0.   0.]
 [  2.  -5.   0.]
 [  0. -10.   0.]] 

	 	 	 	 	 theta[ [2, 3, 4] ] =  1.5707963267948966
g[ [2, 4, 7] ] =  [[ 1.5 12. ]] 	 	 theta[ [2, 4, 7] ] =  1.1902899496825317
g[ [2, 7, 3] ] =  [[71.5 40. ]] 	 	 theta[ [2, 7, 3] ] =  0.3805063771123649
sum( [1.19 0.38] ) =  1.57 
 

GRADIENT: [[[ 1.

Now it comes to effectively calculate the numerical Hamiltonian. As a basic ingredient we define the sample Hamiltonian as the norm of the gradient,
\begin{align*}
    H(U) = H(\nabla U) = \| \nabla U \|_2 = \| U_x + U_y \|_2
\end{align*}
[Is it a function of $U$ or of $\nabla U$?] Going back to the basic and ultimate ingredients, it is a function of $U$. But as the charateristic part of the processing, it is a function of $\nabla U$, so from a mathical point of view we have $H=H(\nabla U)$, but for a betta data structure handle we use $H=H(U)$.

As each equation is associated to a vertex, so each elemental evaluation of a Hamiltonian should be associated to a vertex. Then the generation of the residual as a discretization of the governing equations run over all vertices, calling each single elemental evaluation of a Hamiltonian.


What are the inputs needed such then adaptions can be done just by changing the implementation of the model, but not too many parts of the overall code. The plan is too prepare the overall algorithm as general as possible in order to allow to run over a broad range of model specification without some major need to dig into implemenation details.






In [7]:
# ... should be revised ...

def hamiltonian(U):
    return np.linalg.norm(U)

UU=np.array([[1, 2], [3, 4], [5, 6]])
h=hamiltonian(U)
print(h) 


4.001249804748511


So, the previous implementation is cleaned up by putting all specifics that correspond to a specific vertex into a subroutine, that can be called through an overall routine.

In [65]:
def get_theta_gradient(vh):
    nVertex = vh.idx()
    
    vlist = []
    nlist = []    
    
    for vvh in mesh.vv(vh):
        idx = vvh.idx() # devuelve indice del VERTICE
        vlist.append(idx)
        nlist.append(V[nVertex,:]-V[idx,:])
    nlist=np.array(nlist)
    # print(nVertex,': \n', nlist,'\n')
    
    # for n in nlist:
    THETA=[]
    GRADIENT=[]
    for k in range(len(vlist)):
            v1 = nlist[k-1,:]
            v2 = nlist[k,:]
            theta = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
            I=[nVertex, vlist[k-1], vlist[k]]                
                
            if (abs(theta)<math.pi-1e-3) and (abs(theta) > 1e-3) and (abs(theta-math.pi/2)>1e-3):
                g = getGradent2(I, T[I])
                THETA.append(theta)
                GRADIENT.append(g)
                
    THETA=np.array(THETA)  
    GRADIENT=np.array(GRADIENT)

    return THETA, GRADIENT

print('nVertex:', vh.idx())
theta, gradient = get_theta_gradient(vh)

print('\n sum(', THETA,') = % .2f' % sum(THETA),'\n \n')
print('GRADIENT: \n', GRADIENT,'\n \n')

nVertex: 7

 sum( [0.92 1.03 1.19 0.76 2.38] ) =  6.28 
 

GRADIENT: 
 [[[ 1.5]
  [12. ]]

 [[ 9. ]
  [ 7.5]]

 [[ 9. ]
  [39. ]]

 [[41.5]
  [52. ]]

 [[71.5]
  [40. ]]] 
 



Yet, the assembling of the numerical Hamiltonian should be done in an intermediate subroutine, as there are different choices of how to handle the gradient and angle information.

In [37]:
def ham(vh): 
    theta, gradient = get_theta_gradient(vh)
    gav = gradient.transpose().dot(theta)/sum(theta)
    
    return gav

gav = ham(vh)
print(gav)



[[35.52 31.83]]


This re-organization of the implemenation allows us to call subroutine for the numerical Hamiltonian in a compact way.

In [41]:
Glist = []
for vh in mesh.vertices():
        
    gav = ham(vh)
    Glist.append(gav)
  
    
Glist = np.array(Glist)
print('\n \n -------- List of average gradients ------- \n \n', Glist)



 
 -------- List of average gradients ------- 
 
 [[[ 4.08  2.95]]

 [[30.32 12.79]]

 [[18.46 18.78]]

 [[48.77 49.09]]

 [[ 3.7   7.84]]

 [[31.97 37.53]]

 [[13.49 11.83]]

 [[35.52 31.83]]]


Yet we did not calculate a Hamiltonian different to the identical function that gives an averaged gradient. But with the established procedure, that is now organized in different subroutines, we are in the position to start with the adaptions.

Now, we start to get more involved by defining a local mode parameter, that enables us to control the possibles choices about the Hamiltonian in a systematic way. This saves us from an explosion of different subroutines to handle the Hamiltonian for different choices. Instead the switch to the different choice is done by the mode parameter. This allows a systematic run over the choices in sequence or in parrallel.

The different choices include different options for the numerical Hamiltonian and different modelos of the physical Hamiltonian.




Now, effectively pointing to the considered models, namely
\begin{align*}
   H_T(T, S) = \| \nabla T \| - v = 0 ,
   H_S(T, S) = \frac{\nabla T}{\| \nabla T \|} - \nabla S ,
\end{align*}
we recognize that additional data formats and specifications are required.


The equation
\begin{align*}
    \frac{\nabla T}{\| \nabla T \|} = \nabla S
\end{align*}
actually represents two equations,
where it is not clear what is the transient equation extention, to that
it represents the stationary case. Maybe something like
\begin{align*}
    T_t (1, 1) + \frac{\nabla T}{\| \nabla T \|} = \nabla S
\end{align*}
[One variable, two equations: Can this work?]

Anyway, the two equations are 
\begin{align*}
    \frac{T_x}{\| \nabla T \|} = \nabla S_x \\
    \frac{T_y}{\| \nabla T \|} = \nabla S_y,
\end{align*}
so a numerical Hamiltonian should be designed for each of them.

[In total we have 3 equation for each vertex, but still two variables.]

The issue is the general variable handling.
As the solution variables $T, S$ are used within the calculation of the Hamiltionian,
they should be input parameters of the subroutine.



In [63]:
# def velocity(vh, S):
#    print('should ...')
#    print('...not outsource by now...')
#    return 1

def hamiltonian(vh, T, S):
    nVertex = vh.idx()
    theta, gradient = get_theta_gradient(vh)
    gav = gradient.transpose().dot(theta)/sum(theta)
    
    # print(mode)
    
    if mode =='1':
        H = np.linalg.norm(gav)
    elif mode == 'hamT':
        Tgav = gav
        # H = np.linalg.norm(Tgav) - velocity(vh)
        H = np.linalg.norm(Tgav) - V[nVertex,1]/(S[nVertex]+1e-5)
    elif mode == 'hamS':
        Tgav = gav
        Sgav = gav
        H = Tgav / np.linalg.norm(Tgav) - Sgav
    else:
        'ERR: mode is not defined'
    
    return H




mode = '1'
mode = 'hamT'
mode = 'hamS'

H = hamiltonian(vh, T, S)
print(H)

[[-34.77 -31.16]]


In [64]:
MODES = ['1', 'hamT', 'hamS']

T = getT(V[:,0],V[:,1])
S = getT(V[:,0],V[:,1])

for mode in MODES:
    # print(mode)
    H = hamiltonian(vh, T, S)
    print(mode,': ',H)

1 :  47.69319485752732
hamT :  47.62375042272793
hamS :  [[-34.77 -31.16]]
